In [1]:
import nltk
from nltk import load_parser

In [3]:
sents = ['vos jugás']
grammar = 'integral_grammar.fcfg'
for results in nltk.interpret_sents(sents, grammar):
    for (synrep, semrep) in results:
             print(synrep)

KeyboardInterrupt: 